In [13]:
%matplotlib qt

import numpy as np
import gymnasium
from copy import deepcopy
from math import sqrt, log
from anytree import AnyNode
import matplotlib.pyplot as plt
from gymnasium import spaces

### Sistemas de recompensas

Establecemos 3 sistemas distintos de recompensa:
- Recompensa simple por objetivos alcanzados.
- Recompensa de disatncia.
- Recompensa mixta (combinación de objetivos y distancias).

#### Simple

In [14]:
def calculate_reward_simple(env):
    """
    Calcula la recompensa basada únicamente en objetivos alcanzados y penalizaciones claras.

    Args:
        env (Environment): El entorno del robot (posiciones y estados).

    Returns:
        float: Recompensa acumulada según el estado actual.
    """
    reward = 0

    # Penalización por intentar recoger la pieza en una posición incorrecta.
    if env.has_piece == 0 and env.action_last == 6 and (
        env.robot_x_position != env.piece_x_position or
        env.robot_y_position != env.piece_y_position or
        env.robot_z_position != env.piece_z_position
    ):
        reward -= 50

    # Recompensa por estar donde la pieza.
    if env.robot_x_position == env.piece_x_position and env.robot_y_position == env.piece_y_position and env.robot_z_position == env.piece_z_position and env.has_piece == 0:
        reward += 10

    # Recompensa por recoger la pieza (solo una vez).
    if env.has_piece == 1 and not hasattr(env, 'piece_collected'):
        reward += 100
        setattr(env, 'piece_collected', True)

    # Penalización por abrir/cerrar pinza de forma repetitiva.
    if not hasattr(env, 'pinza_toggle_count'):
        env.pinza_toggle_count = 0

    if env.action_last == 6:
        env.pinza_toggle_count += 1
        if env.pinza_toggle_count > 2: # Penalizar solo si ocurre más de dos veces
            reward -= 500
    else:
        env.pinza_toggle_count = 0  # Reinicia si realiza otra acción

    # Recompensa por progresar hacia el objetivo en los ejes.
    if env.has_piece == 1:
        aligned_axes = sum([
            env.robot_x_position == env.goal_x_position,
            env.robot_y_position == env.goal_y_position,
            env.robot_z_position == env.goal_z_position
        ])
        reward += 50 * aligned_axes

    # Recompensa por estar alineado donde el objetivo mientras lleva la pieza.
    if env.has_piece == 1 and env.robot_x_position == env.goal_x_position and env.robot_y_position == env.goal_y_position and env.robot_z_position == env.goal_z_position:
        reward += 500

    # Recompensa por llevar la pieza al objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 500
        setattr(env, 'goal_reached', True)

    # Recompensa por soltar la pieza en el objetivo.
    if env.has_piece == 0 and env.robot_x_position == env.goal_x_position and env.robot_y_position == env.goal_y_position and env.robot_z_position == env.goal_z_position:
        reward += 5000

    # Penalización por soltar la pieza fuera del objetivo.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 1000
        setattr(env, 'piece_placed', True)

    # Penalización adicional por soltar repetidamente la pieza en posiciones incorrectas.
    if not hasattr(env, 'incorrect_drop_count'):
        env.incorrect_drop_count = 0

    if env.has_piece == 0 and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        env.incorrect_drop_count += 1
        if env.incorrect_drop_count > 1:  # Penalizar solo si ocurre más de una vez
            reward -= 500 * env.incorrect_drop_count
    else:
        env.incorrect_drop_count = 0  # Reiniciar el contador si la pieza está en el lugar correcto

    # Recompensa constante por mantener la pieza.
    if env.has_piece == 1 and not (env.robot_x_position == env.goal_x_position and env.robot_y_position == env.goal_y_position and env.robot_z_position == env.goal_z_position):
        reward += 50

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 50
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    # Penalización por entrar en bucles.
    if hasattr(env, 'recent_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        env.recent_positions.append(current_position)
        # Limitar el historial a las últimas 10 posiciones.
        if len(env.recent_positions) > 10:
            env.recent_positions.pop(0)
        # Penalizar bucles.
        if len(env.recent_positions) > len(set(env.recent_positions)):
            reward -= 200
    else:
        env.recent_positions = [(env.robot_x_position, env.robot_y_position, env.robot_z_position)]

    return reward


#### Distancia

In [15]:
def calculate_reward_distance(env):
    """
    Calcula la recompensa únicamente en función de las distancias al objetivo y la pieza.

    Args:
        env (Environment): El entorno que contiene el estado actual del robot. 

    Returns:
        float: La recompensa calculada para el estado actual del entorno.
    """
    reward = 0

    # Calcular la distancia al objeto
    distance_to_piece = ((env.robot_x_position - env.piece_x_position) ** 2 +
                         (env.robot_y_position - env.piece_y_position) ** 2 +
                         (env.robot_z_position - env.piece_z_position) ** 2) ** 0.5

    # Calcular la distancia al objetivo
    distance_to_goal = ((env.robot_x_position - env.goal_x_position) ** 2 +
                        (env.robot_y_position - env.goal_y_position) ** 2 +
                        (env.robot_z_position - env.goal_z_position) ** 2) ** 0.5

    if env.has_piece == 0:
        # Recompensa basada en la cercanía al objeto
        reward += max(0, 100 - distance_to_piece * 10)
    else:
        # Recompensa basada en la cercanía al objetivo
        reward += max(0, 500 - distance_to_goal * 10)

        # Recompensa adicional por reducir la distancia al objetivo
        if hasattr(env, 'previous_distance_to_goal'):
            if distance_to_goal < env.previous_distance_to_goal:
                reward += 50
        env.previous_distance_to_goal = distance_to_goal

    piece_distance_to_goal = ((env.piece_x_position - env.goal_x_position) ** 2 +
                            (env.piece_y_position - env.goal_y_position) ** 2 +
                            (env.piece_z_position - env.goal_z_position) ** 2) ** 0.5

    # Recompensa por llevar la pieza al objetivo
    if piece_distance_to_goal == 0 and env.has_piece == 0:
        reward += 100000

    return reward


#### Mixed

In [16]:

def calculate_reward_mixed(env):
    """
    Calcula la recompensa mixta basada en objetivos alcanzados y distancia al objetivo. Añade penalizaciones por movimientos redundantes y bucles.

    Args:
        env (Environment): El entorno que contiene el estado actual del robot.

    Returns:
        float: La recompensa calculada para el estado actual del entorno.
    """
    reward = 0 

    if env.has_piece == 0:
        # Recompensa proporcional a la cercanía al objeto.
        distance_to_piece = ((env.robot_x_position - env.piece_x_position) ** 2 +
                             (env.robot_y_position - env.piece_y_position) ** 2 +
                             (env.robot_z_position - env.piece_z_position) ** 2) ** 0.5
        reward += max(0, 50 - distance_to_piece * 2)
    else:
        # Recompensa proporcional a la cercanía al objetivo.
        distance_to_goal = ((env.robot_x_position - env.goal_x_position) ** 2 +
                            (env.robot_y_position - env.goal_y_position) ** 2 +
                            (env.robot_z_position - env.goal_z_position) ** 2) ** 0.5
        reward += max(0, 200 - distance_to_goal * 2)

        # Penalización y recompensa acumulativa basada en la distancia al objetivo.
        if hasattr(env, 'previous_distance_to_goal'):
            if distance_to_goal > env.previous_distance_to_goal:
                reward -= 5000
            else:
                reward += 10000
        env.previous_distance_to_goal = distance_to_goal

    # Recompensa por llegar a la pieza.
    if env.piece_x_position == env.robot_x_position and env.piece_y_position == env.robot_y_position and env.piece_z_position == env.robot_z_position:
        reward += 5000

    # Recompensa por recoger la pieza (solo una vez).
    if env.has_piece == 1 and not hasattr(env, 'piece_collected'):
        reward += 100
        setattr(env, 'piece_collected', True)

    # Recompensa constante por mantener la pieza.
    if env.has_piece == 1:
        reward += 10000

    # Recompensa por llegar al objetivo con la pieza.
    if env.piece_x_position == env.robot_x_position == env.goal_x_position and env.piece_y_position == env.robot_y_position == env.goal_y_position and env.piece_z_position == env.robot_z_position == env.goal_z_position:
        reward += 500000

    # Recompensa por alcanzar el objetivo.
    if (env.piece_x_position == env.goal_x_position and
        env.piece_y_position == env.goal_y_position and
        env.piece_z_position == env.goal_z_position and
        not hasattr(env, 'goal_reached')):
        reward += 1000000
        setattr(env, 'goal_reached', True)

    # Penalización por soltar la pieza en una posición incorrecta.
    if env.has_piece == 0 and not hasattr(env, 'piece_placed') and (
        env.piece_x_position != env.goal_x_position or
        env.piece_y_position != env.goal_y_position or
        env.piece_z_position != env.goal_z_position
    ):
        reward -= 10000
        setattr(env, 'piece_placed', True)

    # Penalización por movimientos redundantes.
    if hasattr(env, 'visited_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        if current_position in env.visited_positions:
            reward -= 50
        else:
            env.visited_positions.add(current_position)
    else:
        env.visited_positions = set([(env.robot_x_position, env.robot_y_position, env.robot_z_position)])

    # Penalización por entrar en bucles.
    if hasattr(env, 'recent_positions'):
        current_position = (env.robot_x_position, env.robot_y_position, env.robot_z_position)
        env.recent_positions.append(current_position)
        # Limitar el historial a las últimas 10 posiciones.
        if len(env.recent_positions) > 10:
            env.recent_positions.pop(0)
        # Penaliar bucles.
        if len(env.recent_positions) > len(set(env.recent_positions)):
            reward -= 20000
    else:
        env.recent_positions = [(env.robot_x_position, env.robot_y_position, env.robot_z_position)]

    return reward

### Definición del entorno

In [17]:
class RobotPickAndPlaceEnv(gymnasium.Env):
    """
    Representa el entorno del robot para resolver el problema de pick & place.

    Este entorno simula un espacio discreto en 3D donde un robot debe recoger una 
    pieza en un punto inicial y transportarla a un objetivo. El robot puede moverse 
    en las direcciones X, Y y Z, además de abrir o cerrar su pinza para coger y soltar la pieza.

    Attributes:
        robot_position: Posición actual del robot en (x, y, z).
        piece_position: Posición actual de la pieza en (x, y, z). (-1, -1, -1 si está siendo transportada).
        goal_position: Posición del objetivo en (x, y, z).
        has_piece (bool): Indica si el robot tiene la pieza agarrada (1 si tiene la pieza, 0 en caso contrario).
        step_limit (int): Número máximo de pasos permitidos.
        action_space (gymnasium.spaces.Discrete): Espacio de acciones (0-6).
        observation_space (gymnasium.spaces.Tuple): Espacio de observaciones del entorno.
        reward_function: Función de recompensa utilizada para evaluar los pasos del robot.
    """
    def __init__(self, reward_function=calculate_reward_simple):
        """
        Inicializa el entorno del robot y establece la función de recompensa, por defecto simple.

        Configura los espacios de acción y observación, y define las posiciones iniciales 
        del robot, la pieza y el objetivo.
        """
        super(RobotPickAndPlaceEnv, self).__init__()

        # Definir espacio de acciones: 0 = mover izquierda, 1 = mover derecha, 2 = mover abajo, 3 = mover arriba, 4 = mover atrás, 5 = mover delante, 6 = abrir/cerrar pinza
        self.action_space = spaces.Discrete(7)

        # Definir espacio de observaciones
        self.observation_space = spaces.Tuple((
            spaces.Discrete(10),  # Posición del robot en el eje X (0-9)
            spaces.Discrete(10),  # Posición del robot en el eje Y (0-9)
            spaces.Discrete(10),  # Posición del robot en el eje Z (0-9)
            spaces.Discrete(2),   # Tiene pieza (0 o 1)
            spaces.Discrete(11),  # Posición de la pieza en el eje X (0-9) (-1 si está siendo transportada)
            spaces.Discrete(11),  # Posición de la pieza en el eje Y (0-9) (-1 si está siendo transportada)
            spaces.Discrete(11)   # Posición de la pieza en el eje Z (0-9) (-1 si está siendo transportada)
        ))

        # Establecer función de recompensa
        self.reward_function = reward_function
        
        # Inicializar estado
        self.reset()

        # Inicializar figura, eje y gráfica de matplotlib
        self.fig = None
        self.ax = None

    def reset(self):
        """
        Reinicia el entorno a su estado inicial.

        Returns:
            tuple: Observación inicial del entorno.
        """
        # Condiciones iniciales
        self.robot_x_position = 0  # El robot comienza en la posición (0,0,0)
        self.robot_y_position = 0
        self.robot_z_position = 0
        self.has_piece = 0         # El robot no tiene la pieza
        self.piece_x_position = 5  # La pieza comienza en la posición (5,5,5)
        self.piece_y_position = 5
        self.piece_z_position = 5
        self.goal_x_position = 9   # El objetivo está en la posición (9, 7, 9)
        self.goal_y_position = 7
        self.goal_z_position = 9
        self.steps = 0             # Contador de pasos por episodio

        return self._get_observation()

    def step(self, action):
        """
        Realiza un paso en el entorno según la acción proporcionada.

        Args:
            action (int): Índice de la acción a realizar (0-6).

        Returns:
            tuple: Estado actual del entorno, recompensa obtenida, 
                indicador de si la tarea ha terminado, indicador de truncado, 
                y diccionario de información adicional.
        """
        # Asegurarse de que la acción es válida
        assert self.action_space.contains(action), "Acción inválida"
        done = False
        truncated = False
        self.action_last = action

        # Realizar movimientos según la acción
        if action == 0:  # Mover izquierda
            self.robot_x_position = max(0, self.robot_x_position - 1)
        elif action == 1:  # Mover derecha
            self.robot_x_position = min(9, self.robot_x_position + 1)
        elif action == 2:  # Mover abajo
            self.robot_y_position = max(0, self.robot_y_position - 1)
        elif action == 3:  # Mover arriba
            self.robot_y_position = min(9, self.robot_y_position + 1)
        elif action == 4:  # Mover atrás
            self.robot_z_position = max(0, self.robot_z_position - 1)
        elif action == 5:  # Mover adelante
            self.robot_z_position = min(9, self.robot_z_position + 1)
        elif action == 6:  # Abrir/cerrar pinza
            if not self.has_piece:
                if (self.robot_x_position == self.piece_x_position and
                    self.robot_y_position == self.piece_y_position and
                    self.robot_z_position == self.piece_z_position):
                    self.has_piece = 1
                    self.piece_x_position = -1
                    self.piece_y_position = -1
                    self.piece_z_position = -1
            else:
                self.has_piece = 0
                self.piece_x_position = self.robot_x_position
                self.piece_y_position = self.robot_y_position
                self.piece_z_position = self.robot_z_position
                if (self.piece_x_position == self.goal_x_position and
                    self.piece_y_position == self.goal_y_position and
                    self.piece_z_position == self.goal_z_position):
                    done = True

        # Calcular recompensa y verificar si se ha alcanzado el objetivo
        reward = self.reward_function(self)
        self.steps += 1
        
        if self.steps >= 1000:  # Límite de pasos por episodio
            truncated = True
        return self._get_observation(), reward, done, truncated, {}

    def render(self):
        """
        Renderiza el estado actual del entorno en una gráfica 3D.
        """
        # Inicializar figura y ejes si no existen
        if self.fig is None or self.ax is None:
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(111, projection="3d")

            # Inicializar gráficos del robot, la pieza y el objetivo
            self.robot_plot, = self.ax.plot([], [], [], "go", label="Robot", markersize=7)  # Punto verde para el robot
            self.goal_plot, = self.ax.plot([], [], [], "ro", label="Objetivo", markersize=8)  # Punto rojo para el objetivo
            self.piece_plot, = self.ax.plot([], [], [], "bo", label="Pieza", markersize=6)  # Punto azul para la pieza

            # Configuración de los límites del eje (ajústalos según tu entorno)
            self.ax.set_xlim([0, 10])
            self.ax.set_ylim([0, 10])
            self.ax.set_zlim([0, 10])

            # Etiquetas y leyenda
            self.ax.set_xlabel("X")
            self.ax.set_ylabel("Y")
            self.ax.set_zlabel("Z")
            self.ax.legend()

        # Actualizar posición del robot
        self.robot_plot.set_data([self.robot_x_position], [self.robot_y_position])
        self.robot_plot.set_3d_properties([self.robot_z_position])

        # Actualizar posición de la pieza
        if self.has_piece:
            self.piece_plot.set_data([self.robot_x_position], [self.robot_y_position])
            self.piece_plot.set_3d_properties([self.robot_z_position])
        else:
            self.piece_plot.set_data([self.piece_x_position], [self.piece_y_position])
            self.piece_plot.set_3d_properties([self.piece_z_position])

        # Actualizar posición del objetivo
        self.goal_plot.set_data([self.goal_x_position], [self.goal_y_position])
        self.goal_plot.set_3d_properties([self.goal_z_position])

        # Actualizar el título con el número de pasos
        self.ax.set_title(f"Paso: {self.steps} | Posición Robot: ({self.robot_x_position}, {self.robot_y_position}, {self.robot_z_position}) | Posición Pieza: ({self.piece_x_position}, {self.piece_y_position}, {self.piece_z_position})")

        # Dibujar y pausar para visualización en tiempo real
        plt.draw()
        plt.pause(0.01)

    def _get_observation(self):
        """
        Obtiene la observación actual del entorno.

        Returns:
            tuple: Estado actual del entorno.
        """
        return (self.robot_x_position, self.robot_y_position, self.robot_z_position, self.has_piece, self.piece_x_position, self.piece_y_position, self.piece_z_position)
    
    def close(self):
        """
        Cierra la visualización del entorno.
        """
        plt.close()

## Implemetación de Monte Carlo Tree Search (MCTS)

### Clase MCTSNode

In [18]:
class MCTSNode(AnyNode):
    """
    Nodo para MCTS que hereda de AnyNode.
    La relación padre-hijo es gestionada por AnyTree:
      - self.parent = nodo padre
      - self.children = conjunto de hijos
    """
    def __init__(
        self,
        env,
        state,
        parent=None,
        action_from_parent=None,
        reward_from_parent=0.0,
        done=False,
        c_value=1.4
    ):
        super().__init__(parent=parent)
        self.env = env
        self.state = state
        self.action_from_parent = action_from_parent
        self.reward_from_parent = reward_from_parent  # <--- Guardar la recompensa recibida al llegar a este nodo
        self.done = done
        self.c_value = c_value

        # MCTS stats
        self.Q = 0.0
        self.N = 0

        # Para expandir acciones
        self.remaining_actions = list(range(env.action_space.n))
        self.fully_expanded = False


    def is_leaf(self):
        """
        Un nodo se considera 'hoja' si está en estado terminal (done),
        o bien si no tiene hijos después de haber sido visitado alguna vez.
        (es decir, no se ha expandido o ya se expandió y no hay más acciones).
        """
        return self.done or (len(self.children) == 0 and self.N > 0)

### Clase MCTS

In [19]:
class MCTS:
    """
    Clase principal de MCTS que construye un árbol usando AnyNode.
    Funciona en cualquier entorno Gym (Discrete) siempre que podamos clonar su estado.
    """
    def __init__(self, root_env, c_value=1.4, max_depth=50, rollout_policy="random"):
        """
        root_env: entorno Gym (o Gymnasium).
                  Se usará para inicializar el nodo raíz.
        c_value: constante de exploración UCB.
        max_depth: profundidad máxima del rollout en simulate().
        rollout_policy: define la estrategia de simulación ("random", "heuristic", etc.).
        """
        # Reiniciamos el entorno raíz para obtener la observación inicial
        obs = root_env.reset()

        # Creamos el nodo raíz con un clon de root_env
        self.root = MCTSNode(
            env=deepcopy(root_env),
            state=obs,
            parent=None,
            done=False,
            c_value=c_value
        )
        self.c_value = c_value
        self.max_depth = max_depth
        self.rollout_policy = rollout_policy

    def optimize(self, n_iter=1000):
        """
        Ejecuta el bucle de MCTS n_iter veces (each iteration = select -> expand -> simulate -> backprop).
        """
        for _ in range(n_iter):
            # 1) Selección
            node = self.select(self.root)
            # 2) Expansión
            leaf = self.expand(node)
            # 3) Simulación
            value = self.simulate(leaf)
            # 4) Retropropagación
            self.backpropagate(leaf, value)

    def select(self, node):
        """
        Navega el árbol desde 'node' eligiendo hijos vía UCB1 hasta encontrar
        un nodo que no esté totalmente expandido o sea hoja.
        """
        while not node.done and node.fully_expanded and len(node.children) > 0:
            node = self._best_child_ucb1(node)
        return node

    def _best_child_ucb1(self, node):
        """
        Dado un nodo, retorna el hijo con mayor valor UCB1.
        UCB1 = (Q / N) + c * sqrt(log(N_parent) / N_child)
        """
        best_value = float("-inf")
        best_child = None

        for child in node.children:
            if child.N == 0:
                # Evitar división por cero
                ucb = float("inf")
            else:
                exploit = child.Q / child.N
                explore = self.c_value * sqrt(log(node.N) / child.N)
                ucb = exploit + explore

            if ucb > best_value:
                best_value = ucb
                best_child = child

        return best_child

    def expand(self, node):
        if node.done:
            return node

        actions = node.remaining_actions
        if len(actions) == 0:
            return node

        if self.rollout_policy == "heuristic":
            action = heuristic_policy(node.env)
            if action not in actions:
                action = np.random.choice(actions)
        else:
            action = np.random.choice(actions)

        actions.remove(action)
        if len(actions) == 0:
            node.fully_expanded = True

        env_copy = deepcopy(node.env)
        obs, reward, done, truncated, info = env_copy.step(action)

        child_node = MCTSNode(
            env=env_copy,
            state=obs,
            parent=node,
            action_from_parent=action,
            reward_from_parent=reward,
            done=(done or truncated),
            c_value=node.c_value
        )
        return child_node

    def simulate(self, node):
        """
        Realiza un rollout a partir de 'node' hasta 'max_depth' o estado terminal.
        Retorna la suma de recompensas.
        """
        if node.done:
            return 0.0

        simulation_env = deepcopy(node.env)
        total_reward = 0.0
        depth = 0

        while depth < self.max_depth:
            # Elige acción según la política de rollout
            action = self._rollout_policy(simulation_env)
            obs, reward, done, truncated, info = simulation_env.step(action)
            total_reward += reward
            depth += 1
            if done or truncated:
                break

        return total_reward

    def _rollout_policy(self, env):
        """
        Define la política usada en la fase de simulación/rollout.
        """
        if self.rollout_policy == "random":
            return env.action_space.sample()
        elif self.rollout_policy == "heuristic":
            return heuristic_policy(env)
        else:
            # Por defecto, random si no se reconoce la política
            return env.action_space.sample()

    def backpropagate(self, node, value):
        """
        Propaga el valor de la simulación hacia la raíz,
        actualizando Q y N en cada nodo del camino.
        """
        current = node
        while current is not None:
            current.N += 1
            current.Q += (current.reward_from_parent + value)
            current = current.parent

    def best_action(self):
        """
        Retorna la acción del hijo de la raíz que tenga más visitas N.
        """
        if len(self.root.children) == 0:
            # Si no hay hijos, devolvemos una acción aleatoria
            return np.random.choice(self.root.env.action_space.n)
        best_child = max(self.root.children, key=lambda c: c.N)
        return best_child.action_from_parent

    def re_root(self, action):
        """
        Hace 're-rooting' de la raíz al hijo que surge de 'action'.
        Si no existe el hijo, no hace nada.
        """
        matching_children = [c for c in self.root.children if c.action_from_parent == action]
        if not matching_children:
            return
        new_root = matching_children[0]
        new_root.parent = None  # AnyTree: quitamos la relación con la antigua raíz
        self.root = new_root

### Heurística

In [20]:
def heuristic_policy(env):
    rx, ry, rz, has_piece, px, py, pz = (
        env.robot_x_position, env.robot_y_position, env.robot_z_position,
        env.has_piece, env.piece_x_position, env.piece_y_position, env.piece_z_position
    )
    gx, gy, gz = env.goal_x_position, env.goal_y_position, env.goal_z_position

    if has_piece == 0:
        # Si está en la pieza, recogerla
        if (rx, ry, rz) == (px, py, pz):
            return 6  # Cerrar pinza
        
        # Priorizar el movimiento hacia el eje con mayor distancia
        distances = [
            (abs(rx - px), 1 if rx < px else 0),
            (abs(ry - py), 3 if ry < py else 2),
            (abs(rz - pz), 5 if rz < pz else 4),
        ]
        return max(distances, key=lambda x: x[0])[1]

    else:
        # Si está en el objetivo, soltar la pieza
        if (rx, ry, rz) == (gx, gy, gz):
            return 6  # Soltar pinza
        
        # Priorizar el movimiento hacia el eje con mayor distancia al objetivo
        distances = [
            (abs(rx - gx), 1 if rx < gx else 0),
            (abs(ry - gy), 3 if ry < gy else 2),
            (abs(rz - gz), 5 if rz < gz else 4),
        ]
        return max(distances, key=lambda x: x[0])[1]



## Robot Pick and Place

### Renderizado

In [21]:
def run_mcts_robot(env, c_value=1.4, n_iter_mcts=1000, rollout_policy="random", max_steps=100, max_depth=150):
    """
    Ejecuta MCTS para un episodio en RobotPickAndPlaceEnv.
    """
    obs = env.reset()  # O (obs, info) si tu reset lo maneja así
    mcts = MCTS(root_env=env, c_value=c_value, max_depth=max_depth, rollout_policy=rollout_policy)

    done = False
    total_reward = 0.0
    step_count = 0

    while not done and step_count < max_steps:
        # 1) MCTS
        mcts.optimize(n_iter=n_iter_mcts)

        # 2) Seleccionar la mejor acción
        action = mcts.best_action()

        # 3) Paso real
        obs, reward, done, truncated, info = env.step(action)
        #print(f"Step={step_count}, Action={action}, State={obs}, Reward={reward}, Done={done}")
        total_reward += reward
        step_count += 1

        # 4) re-root
        mcts.re_root(action)

        # Render si quieres ver la evolución 3D (comentado para no abrir ventanas en cada paso)
        env.render()

    return total_reward, step_count, done


### Análisis de resultados

In [22]:
def sensitivity_study_robot(reward_function=calculate_reward_simple):
    c_values = [0.5, 1.4, 2.0]
    n_iters = [1000, 3000]
    policies = ["random", "heuristic"]
    n_runs = 3

    results = []
    for c_val in c_values:
        for n_it in n_iters:
            for pol in policies:
                rews = []
                steps_list = []
                for _ in range(n_runs):
                    env = RobotPickAndPlaceEnv(reward_function=reward_function)
                    r, s, done = run_mcts_robot(env, c_value=c_val,
                                          n_iter_mcts=n_it,
                                          rollout_policy=pol,
                                          max_steps=150)
                    rews.append(r)
                    steps_list.append(s)
                    if done:
                        break
                avg_r = np.mean(rews)
                avg_s = np.mean(steps_list)
                results.append((c_val, n_it, pol, avg_r, avg_s))
                print(f"Sensibilidad Robot -> c={c_val}, n={n_it}, pol={pol} -> reward={avg_r:.1f}, steps={avg_s:.1f}, done={done}")

    return results

def sensitivity_study_robot_single_run(reward_function=calculate_reward_simple):
    c_values = [0.5, 1.4, 2.0]
    n_iters = [1000, 3000]
    policies = ["random", "heuristic"]

    results = []
    for c_val in c_values:
        for n_it in n_iters:
            for pol in policies:
                env = RobotPickAndPlaceEnv(reward_function=reward_function)
                r, s, done = run_mcts_robot(env, c_value=c_val,
                                            n_iter_mcts=n_it,
                                            rollout_policy=pol,
                                            max_steps=100)
                results.append((c_val, n_it, pol, r, s, done))
                print(f"Sensibilidad Robot -> c={c_val}, n={n_it}, pol={pol} -> reward={r:.1f}, steps={s:.1f}, done={done}")
    
    return results

In [23]:
def plot_all_sensitivity_results(results):
    """
    Genera una gráfica que muestra el impacto de los parámetros `c`, `n` y `p` en los resultados,
    incluyendo marcadores para indicar cuándo se alcanzó `done=True`.
    
    Args:
        results (list): Lista de tuplas (c, n, p, avg_r, avg_s, done).
    """
    policies = list(set([p for _, _, p, _, _, _ in results]))
    iterations = list(set([n for _, n, _, _, _, _ in results]))
    c_values = sorted(set([c for c, _, _, _, _, _ in results]))
    
    fig, axs = plt.subplots(len(policies), len(iterations), figsize=(15, 10), sharex=True, sharey=True)
    
    for i, pol in enumerate(policies):
        for j, n in enumerate(iterations):
            ax = axs[i, j]
            filtered_results = [(c, avg_r, done) for c, n_iter, p, avg_r, _, done in results if p == pol and n_iter == n]
            c_vals, avg_rewards, dones = zip(*filtered_results)
            
            ax.plot(c_vals, avg_rewards, label=f'n={n}, {pol}')
            done_indices = [k for k, d in enumerate(dones) if d]
            if done_indices:
                ax.scatter(
                    [c_vals[k] for k in done_indices],
                    [avg_rewards[k] for k in done_indices],
                    color="black", label="Done"
                )
            ax.set_title(f'Pol: {pol}, n: {n}')
            ax.set_xlabel("c")
            ax.set_ylabel("Reward")
            ax.legend()
            ax.grid()
    
    fig.suptitle("Impacto de la constante c en el robot para distintas políticas e iteraciones")
    plt.tight_layout()
    plt.show()


### Entrenamiento y evaluación

In [ ]:
results = sensitivity_study_robot_single_run(reward_function=calculate_reward_mixed)
# results = sensitivity_study_robot(reward_function=calculate_reward_simple)

In [244]:
plot_all_sensitivity_results(results)

## Frozen Lake

### Renderizado

In [24]:
def run_mcts_frozenlake(is_slippery=False, c_value=1.4, n_iter_mcts=1000, rollout_policy="random"):
    env = gymnasium.make('FrozenLake-v1', is_slippery=is_slippery, render_mode=None)
    obs, _info = env.reset()

    # Creamos MCTS con el nodo raíz
    mcts = MCTS(
        root_env=env,
        c_value=c_value,
        max_depth=50,
        rollout_policy=rollout_policy
    )

    done = False
    total_reward = 0
    step_count = 0
    max_steps = 100

    while not done and step_count < max_steps:
        # 1) Ejecutar MCTS
        mcts.optimize(n_iter=n_iter_mcts)

        # 2) Seleccionar la mejor acción
        action = mcts.best_action()

        # 3) Paso en el entorno real
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        step_count += 1

        # 4) Re-root
        mcts.re_root(action)

    env.close()
    return total_reward, step_count, done

### Entrenamiento y evaluación

In [ ]:
r, steps, done = run_mcts_frozenlake(is_slippery=False, c_value=2.0, n_iter_mcts=2000, rollout_policy="random")
print(f"Resultado FrozenLake: Done = {done}, Reward = {r}, Steps = {steps}")

In [ ]:
r, steps, done = run_mcts_frozenlake(is_slippery=True, c_value=1.0, n_iter_mcts=2000, rollout_policy="random")
print(f"Resultado FrozenLake: Done = {done}, Reward = {r}, Steps = {steps}")